In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/isemeval-dataset/iSarcasmEval-main/LICENSE
/kaggle/input/isemeval-dataset/iSarcasmEval-main/README.md
/kaggle/input/isemeval-dataset/iSarcasmEval-main/third-party annotations/README.md
/kaggle/input/isemeval-dataset/iSarcasmEval-main/third-party annotations/arabic_task_a.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/third-party annotations/english_task_a.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/third-party annotations/arabic_task_c.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/third-party annotations/english_task_c.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/test/task_B_En_test.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/test/task_A_Ar_test.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/test/task_C_Ar_test.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/test/task_C_En_test.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/test/task_A_En_test.csv
/kaggle/input/isemeval-dataset/iSarcasmEval-main/train/train.En.csv
/k

In [2]:
# Install libraries
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [3]:
# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

# Hugging Face Dataset
from datasets import Dataset

# Model performance evaluation
import evaluate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import csv

# Define the CSV file path
csv_file = '/kaggle/input/isemeval-dataset/iSarcasmEval-main/test/task_A_En_test.csv'

# Initialize an empty list to store the data
data_test = []

# Open the CSV file
with open(csv_file, mode='r', newline='') as file:
    # Create a CSV reader
    reader = csv.reader(file)

    # Skip the header row if it exists (optional)
    header = next(reader)

    # Iterate over the rows in the CSV file
    for row in reader:
        data_test.append(row)


In [5]:
data_test[0:2]

[["Size on the the Toulouse team, That pack is monstrous. \n\nI can't see a Welsh region ever winning this, Money talks as they say .",
  '0'],
 ['Pinball!', '0']]

In [6]:
data_test = pd.DataFrame(data_test)
# Assuming 'data_test' is your DataFrame
data_test = data_test.rename(columns={0: 'headline', 1: 'label'})

# Now, your DataFrame columns are renamed


In [7]:
data_test.head()

,headline,label
0,"Size on the the Toulouse team, That pack is mo...",0
1,Pinball!,0
2,So the Scottish Government want people to get ...,1
3,villainous pro tip : change the device name on...,0
4,I would date any of these men 🥺,0


In [8]:
import csv

# Define the CSV file path
csv_file = '/kaggle/input/isemeval-dataset/iSarcasmEval-main/train/train.En.csv'

# Initialize an empty list to store the data
data_train = []

# Open the CSV file
with open(csv_file, mode='r', newline='') as file:
    # Create a CSV reader
    reader = csv.reader(file)

    # Skip the header row if it exists (optional)
    header = next(reader)

    # Iterate over the rows in the CSV file
    for row in reader:
        # Append only the first two values (columns) from each row
        data_train.append(row[1:3])

# Now, 'data_test' contains the first two columns of the CSV data


In [9]:
data_train = pd.DataFrame(data_train)
# Assuming 'data_test' is your DataFrame
data_train = data_train.rename(columns={0: 'headline', 1: 'label'})

In [10]:
# Assuming you've already loaded your CSV data into a DataFrame 'data_test'

# Convert the 'label' column to integers
data_test['label'] = data_test['label'].astype(int)
data_train['label'] = data_train['label'].astype(int)


In [11]:
data_train.head()

,headline,label
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1


In [12]:
data_train['label'].value_counts()

label
0    2601
1     867
Name: count, dtype: int64

In [13]:
data_test['label'].value_counts()

label
0    1200
1     200
Name: count, dtype: int64

In [14]:
# Install libraries
# !pip install transformers datasets evaluate

In [15]:
# Convert pyhton dataframe to Hugging Face arrow dataset
hg_train_data = Dataset.from_pandas(data_train)
hg_test_data = Dataset.from_pandas(data_test)

In [16]:
# Length of the Dataset
print(f'The length of hg_train_data is {len(hg_train_data)}.\n')

# Check one review
hg_train_data[0]

The length of hg_train_data is 3468.



{'headline': 'The only thing I got from college is a caffeine addiction',
 'label': 1}

In [17]:
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Take a look at the tokenizer
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [18]:
# Mapping between special tokens and their IDs.
print(f'The unknown token is {tokenizer.unk_token} and the ID for the unkown token is {tokenizer.unk_token_id}.')
print(f'The seperator token is {tokenizer.sep_token} and the ID for the seperator token is {tokenizer.sep_token_id}.')
print(f'The pad token is {tokenizer.pad_token} and the ID for the pad token is {tokenizer.pad_token_id}.')
print(f'The sentence level classification token is {tokenizer.cls_token} and the ID for the classification token is {tokenizer.cls_token_id}.')
print(f'The mask token is {tokenizer.mask_token} and the ID for the mask token is {tokenizer.mask_token_id}.')

The unknown token is [UNK] and the ID for the unkown token is 100.
The seperator token is [SEP] and the ID for the seperator token is 102.
The pad token is [PAD] and the ID for the pad token is 0.
The sentence level classification token is [CLS] and the ID for the classification token is 101.
The mask token is [MASK] and the ID for the mask token is 103.


In [19]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["headline"], 
                     max_length=32, 
                     truncation=True, 
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

  0%|          | 0/3468 [00:00<?, ?ex/s]

  0%|          | 0/1400 [00:00<?, ?ex/s]

In [20]:
# Take a look at the data
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['headline', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3468
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1400
})


In [21]:
dataset_train[0]

{'headline': 'The only thing I got from college is a caffeine addiction',
 'label': 1,
 'input_ids': [101,
  1109,
  1178,
  1645,
  146,
  1400,
  1121,
  2134,
  1110,
  170,
  11019,
  15475,
  2042,
  15658,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [22]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",          
    logging_dir='./sentiment_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=3,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,  
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [24]:
# Number of evaluation modules
print(f'There are {len(evaluate.list_evaluation_modules())} evaluation models in Hugging Face.\n')

# List all evaluation metrics
evaluate.list_evaluation_modules()

There are 140 evaluation models in Hugging Face.



['precision',
 'code_eval',
 'roc_auc',
 'cuad',
 'xnli',
 'rouge',
 'pearsonr',
 'mse',
 'super_glue',
 'comet',
 'cer',
 'sacrebleu',
 'mahalanobis',
 'wer',
 'competition_math',
 'f1',
 'recall',
 'coval',
 'mauve',
 'xtreme_s',
 'bleurt',
 'ter',
 'accuracy',
 'exact_match',
 'indic_glue',
 'spearmanr',
 'mae',
 'squad',
 'chrf',
 'glue',
 'perplexity',
 'mean_iou',
 'squad_v2',
 'meteor',
 'bleu',
 'wiki_split',
 'sari',
 'frugalscore',
 'google_bleu',
 'bertscore',
 'matthews_correlation',
 'seqeval',
 'trec_eval',
 'rl_reliability',
 'angelina-wang/directional_bias_amplification',
 'cpllab/syntaxgym',
 'kaggle/ai4code',
 'codeparrot/apps_metric',
 'mfumanelli/geometric_mean',
 'poseval',
 'brier_score',
 'abidlabs/mean_iou',
 'abidlabs/mean_iou2',
 'giulio98/codebleu',
 'mase',
 'mape',
 'smape',
 'dvitel/codebleu',
 'NCSOFT/harim_plus',
 'JP-SystemsX/nDCG',
 'Drunper/metrica_tesi',
 'jpxkqx/peak_signal_to_noise_ratio',
 'jpxkqx/signal_to_reconstruction_error',
 'hpi-dhc/FairEva

In [25]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

UsageError: api_key not configured (no-tty). call wandb.login(key=[your_api_key])

In [ ]:
trainer.evaluate(dataset_test)

In [ ]:
# Save tokenizer
tokenizer.save_pretrained('./sentiment_transfer_learning_transformer_pre/')

# Save model
trainer.save_model('./sentiment_transfer_learning_transformer_pre/')